In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bsp

1.爬取学校网站

In [3]:
html = urlopen("http://news.cufe.edu.cn/jdxw/104361.htm")
bs = bsp(html.read(),'html.parser')
titlelist = bs.findAll('div', {'class':'titop'})
for title in titlelist:
    print(title.get_text())


		  校党委书记何秀超一行拜访海南省长沈晓明
		  




In [4]:
textList = bs.findAll('div', {'class':'neir'})
for text in textList:
    print(text.get_text())


　　2018年4月13日，习近平总书记在庆祝海南建省办经济特区30周年大会上发表重要讲话，宣布党中央决定支持海南全岛建设自由贸易试验区，支持海南逐步探索、稳步推进中国特色自由贸易港建设，为海南未来发展指明了战略方向。在总书记发表“413”重要讲话一周年之际，为贯彻落实总书记讲话精神，践行学校“走出去”战略，校党委书记何秀超于近日带队赴海南洽商合作事宜。
　　4月18上午，何秀超书记一行应邀与海南省委副书记、省长沈晓明举行会谈，就省校合作交换意见，探讨交流具体合作项目。海南省副省长王路、省政府秘书长倪强和省教育厅厅长曹献坤参加会谈。我校副校长史建平，学校办公室、国际合作处、港澳台事务办公室和合作发展办公室的负责同志一同参加会谈。
　　会上，沈晓明省长对何秀超书记一行到海南洽谈合作表示热烈欢迎，对学校积极推动“走出去”战略表示赞赏。沈省长指出，当前海南全省上下正在全面贯彻落实习总书记“413”重要讲话精神，全面深化改革开放，自贸区建设实现了良好开局。沈省长表示，中央财经大学是“中国财经管理专家的摇篮”，具有突出的学科优势和雄厚的高端人才储备，学校提出的合作意向十分契合总书记对海南的要求，符合海南未来发展战略，省校合作前景非常广阔。沈省长要求，省政府有关主管领导和相关部门要确定专人与学校对接，大力推进省校合作有关事宜。
　　何秀超书记对海南省的热情接待表示感谢。何书记指出，中财大经过近70年的发展，形成了鲜明的办学特色，学校的优势与海南省自贸区建设的发展急需十分吻合，学校高度重视与海南省的合作，已经做好了为海南省经济社会发展服务的充足准备。何书记表示，期盼省委、省政府多关注中财大的发展，支持学校建设；学校将全力以赴，充分发挥学科建设、人才培养、科研开发及成果转化等方面的优势，支持海南经济社会和高等教育改革发展，为海南自贸区建设服好务，期待双方的合作能够实现互利共赢，结出硕果。
　　会后，何秀超书记、史建平副校长和王路副省长、曹献坤厅长随即进行了进一步交流。

合影



2.爬取百度新闻

In [11]:
import pandas as pd
import logging
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from faker import Faker


def get_list(href):
    
    """    Function to get  web list pages for a given url.    """
    
    news = pd.DataFrame()
    
    fake = Faker()
    ua = fake.user_agent()
    headers={"User-Agent":ua}  #请求头
    html = requests.get(href,headers=headers)
    
    # Parsing html
    soup = BeautifulSoup(html.content, 'html.parser')
    divs = soup.findAll('div', {"class": "result"})
    for div in divs:
        head = div.findAll('h3',{"class":"c-title"})[0]
        # News title
        title = head.get_text().replace('\n','')
        # News url
        url = head.find('a')['href']
        # Other info
        otherinfo = div.findAll('div',{"class":"c-summary c-row"})[0].get_text().split()
        source,date,time = otherinfo[:3]
        abstract = otherinfo[3:]
        
        total = {'Date':date,'Time':time,'Source':source,'Title':title,'Abstract':[abstract],'URL':url}
        news = news.append(pd.DataFrame(total))
    return news

if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)
    compRawStr = '中央财经大学'
    # Dealing with character encoding
    comp = compRawStr.encode('gbk')
    d = {'word': comp}
    pname = urlencode(d)
    # Scraping and printing the first two pages
    newsData = pd.DataFrame()
    for page in range(1,3):
        href = 'http://www.baidu.com/s?rtt=1&bsst=1&cl=2&tn=news&rsv_dl=ns_pc&%s&tngroupname=organic_news&pn=%s'% (pname,(page-1)*10)
        newsData = newsData.append(get_list(href))
        logging.info('Scraping ' + href)

INFO:root:Scraping http://www.baidu.com/s?rtt=1&bsst=1&cl=2&tn=news&rsv_dl=ns_pc&word=%D6%D0%D1%EB%B2%C6%BE%AD%B4%F3%D1%A7&tngroupname=organic_news&pn=0
INFO:root:Scraping http://www.baidu.com/s?rtt=1&bsst=1&cl=2&tn=news&rsv_dl=ns_pc&word=%D6%D0%D1%EB%B2%C6%BE%AD%B4%F3%D1%A7&tngroupname=organic_news&pn=10


In [12]:
newsData.reset_index(drop = True)

,Date,Time,Source,Title,Abstract,URL
0,2019年03月06日,21:33,田田看教育,"这所大学竟然超过了中央财经大学,排在了北京高校前五强","[要说中央财经大学可是中国首屈一指的财类巨头,还是被北航抢尽了风头,中央财经大学在北京在区都...",https://baijiahao.baidu.com/s?id=1627262964834...
1,2019年01月08日,15:06,博雅数据库,"中央财经大学毕业生都去哪儿了?就业50%留北京,月薪6922元","[中央财经大学2018年共有毕业生:4089人。男女比例1:1.69,少数民族12.3%。,...",https://baijiahao.baidu.com/s?id=1622074869827...
2,2019年01月04日,11:14,小谭论美,"中央财经大学硕士遭人“鄙视”,公司强行辞退,只因毕业于普本","[一位中央财经大学硕士学位的毕业生,在找到工作之后,被强行辞退,只因为这名学生毕业于普通本科...",https://baijiahao.baidu.com/s?id=1621697805310...
3,2018年12月02日,19:15,每日经济新闻,中央财经大学中国企业研究中心研究员刘姝威:AMC应做好不良资产...,"[这种背景之下,中央财经大学中国企业研究中心研究员刘姝威表示,在不良资产处置这个方面,AMC...",https://baijiahao.baidu.com/s?id=1618738418514...
4,2018年11月23日,11:01,记忆里的歌,"中央财经大学,有2个校区,真实的中央财经大学是什么样子的?","[中央财经大学,有2个校区,一个在学院路,就是校本部了;一个在沙河,就是分校区了。这两个校区...",https://baijiahao.baidu.com/s?id=1617891946125...
5,2018年10月01日,16:22,优学优考策略,"上海财经大学和中央财经大学相比,哪所大学实力更强?","[上海财经大学和中央财经大学大学相比,哪所大学实力更强?上财,财大人无论学什么专业最后都是会...",https://baijiahao.baidu.com/s?id=1613110509646...
6,2018年09月28日,18:36,苗苗聊教育,"2018年中国财经类大学排名出炉,第一名不是中央财经大学,你服吗",[据2018年国内财经类大学综合实力评比排名来看:上海财经大学排名第一、中南财经政法大学排名...,https://baijiahao.baidu.com/s?id=1612847186896...
7,2018年09月15日,06:14,校园教育,中央财经大学校长在2018年开学典礼上的讲话(全文)!,"[中央财经大学作为新中国成立后中央人民政府直接创办的、与共和国同龄的第一所新型财经高校,与共...",https://baijiahao.baidu.com/s?id=1611592891203...
8,2018年08月17日,16:19,金榜志愿号,"中央财经大学:2018年各专业录取分数,透露出哪些报考“密码”","[中央财经大学号称“中国财经管理专家的摇篮”,“财经院校中的黄埔军校”,是著名的“两财一贸”...",https://baijiahao.baidu.com/s?id=1609030844351...
9,2018年08月14日,15:40,逸凡说考研,2018中央财经大学801经济学学长考研经验(含详细复习计划),"[今天是8.14日,距离2019考研还有130天的时间,在此致硕考研小编为今年备考中央财经大...",https://baijiahao.baidu.com/s?id=1608759222811...


3.爬取新浪新闻(通过设置headers,cookies,time.sleep进行伪装)

In [41]:
import pandas as pd
import logging
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from faker import Faker


def get_list(href):
    
    """    Function to get  web list pages for a given url.    """
    
    news = pd.DataFrame()
    
    fake = Faker()
    ua = fake.user_agent()
    headers={"User-Agent":ua,'Connection': 'keep-alive'}  #请求头
    
    cookie = {"SUB":"_2AkMrnNx7f8NxqwJRmfoVyGPlbIh_zQ3EieKdwC2gJRMyHRl-yD9kql1StRB6ABzylIOGjl6xANXyDkDxcAMmzu4e1p7-","mYSeArcH":"%u4EAC%u4E1C%7CsEaRchHIS%7C%7CsEaRchHIS%7C%u767E%u5EA6%7CsEaRchHIS%7Cj%u4EAC%u4E1C%7CsEaRchHIS%7C%u5C71%u5927%7CsEaRchHIS%7C%u5C71%u4E1C%u5927%u5B66%7CsEaRchHIS%7C%u4E2D%u592E%u8D22%u7ECF%u5927%u5B66%7CsEaRchHIS%7C%u6D77%u519B%7CsEaRchHIS%7C%u4E94%u4E00%7CsEaRchHIS%7C%u4E2D%u592E%u8D22%u7ECF"}

    html = requests.get(href, headers=headers, cookies=cookie)

    
    # Parsing html
    soup = BeautifulSoup(html.content, 'html.parser')
    divs = soup.findAll('div', {"class": "box-result clearfix"})
    for div in divs:
        head = div.findAll('h2')[0]
        
        titleinfo = head.find('a')
        # News title
        title = titleinfo.get_text()
        # News url
        url = titleinfo['href']
        
        # Other info
        otherinfo = head.find('span', {"class": "fgray_time"}).get_text()
        source, date, time = otherinfo.split()
        
        # News abstract
        abstract = div.find('p', {"class": "content"}).get_text()
        
        
        total = {'Date':date,'Time':time,'Source':source,'Title':title,'Abstract':[abstract],'URL':url}
        news = news.append(pd.DataFrame(total))
    return news

if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)
    compRawStr = '中央财经大学'
    # Dealing with character encoding
    comp = compRawStr.encode('gbk')
    d = {'q': comp}
    pname = urlencode(d)
    # Scraping and printing the first two pages
    newsData = pd.DataFrame()
    for page in range(1,4):
        href = 'http://search.sina.com.cn/?%s&c=news&from=index&col=&range=&source=&country=&size=&time=&a=&page=%s&pf=0&ps=0&dpc=1'% (pname,page)
        newsData = newsData.append(get_list(href))
        logging.info('Scraping ' + href)
        time.sleep(5)

INFO:root:Scraping http://search.sina.com.cn/?q=%D6%D0%D1%EB%B2%C6%BE%AD%B4%F3%D1%A7&c=news&from=index&col=&range=&source=&country=&size=&time=&a=&page=1&pf=0&ps=0&dpc=1
INFO:root:Scraping http://search.sina.com.cn/?q=%D6%D0%D1%EB%B2%C6%BE%AD%B4%F3%D1%A7&c=news&from=index&col=&range=&source=&country=&size=&time=&a=&page=2&pf=0&ps=0&dpc=1
INFO:root:Scraping http://search.sina.com.cn/?q=%D6%D0%D1%EB%B2%C6%BE%AD%B4%F3%D1%A7&c=news&from=index&col=&range=&source=&country=&size=&time=&a=&page=3&pf=0&ps=0&dpc=1


In [42]:
newsData.reset_index(drop = True)

,Date,Time,Source,Title,Abstract,URL
0,2019-05-09,15:24:22,新浪教育,王牌对王牌 相似王牌专业高校谁更胜一筹？,而且单单从王牌专业的排名上来看 成电也是要多于西电的 中央财经大学VS上海财经大...,http://edu.sina.com.cn/gaokao/2019-05-09/doc-i...
1,2019-05-09,11:46:00,界面,人民网：二十八条措施为养老服务通堵点消痛点,我国养老服务无论是数量上还是质量上都存在着不充分、不平衡的问题 ”中央财经大学教授刘钧说 养...,https://k.sina.com.cn/article_6192937794_17120...
2,2019-05-09,11:46:00,人民网,人民网：二十八条措施为养老服务通堵点消痛点,我国养老服务无论是数量上还是质量上都存在着不充分、不平衡的问题 ”中央财经大学教授刘钧说 养...,https://finance.sina.com.cn/roll/2019-05-09/do...
3,2019-05-09,11:37:43,财经全记录,首推权益商城创新用户增值服务 邮储银行携京东升级6亿用户体验,今年2月 中国银行保险监督管理委员会首次将邮储银行机构类型调整为“国有大型商业银行” 与...,https://t.cj.sina.com.cn/articles/view/6585238...
4,2019-05-09,10:47:58,北京学为贵,福利 | 完全掌握这197个同替词汇，你的雅思听力就能至少提1分!,讲师 学为贵雅思听力名师 10年雅思教学经验 曾任教于清华大学...,https://k.sina.com.cn/article_3080457541_b79c0...
5,2019-05-09,00:53:53,北京商报,火车站售票窗口频现“被保险” 中铁自保3元意外险遭质疑,火车站售票窗口频现“被保险” 中铁自保3元意外险遭质疑 网购火车票...,https://finance.sina.com.cn/roll/2019-05-09/do...
6,2019-05-09,00:14:37,中国证券报,永赢基金管理有限公司永赢嘉益债券型证券投资基金更新招募说明书摘要,永赢基金管理有限公司永赢嘉益债券型证券投资基金更新招募说明书摘要 中国证券报 ...,https://finance.sina.com.cn/roll/2019-05-09/do...
7,2019-05-09,00:14:36,中国证券报,北信瑞丰健康生活主题灵活配置混合型证券投资基金招募说明书（更新）摘要,北信瑞丰健康生活主题灵活配置混合型证券投资基金招募说明书（更新）摘要 中国证券报 ...,https://finance.sina.com.cn/roll/2019-05-09/do...
8,2019-05-09,00:14:35,中国证券报,上证180公司治理交易型开放式指数证券投资基金（更新）招募说明书摘要,上证180公司治理交易型开放式指数证券投资基金（更新）招募说明书摘要 中国证券报 ...,https://finance.sina.com.cn/roll/2019-05-09/do...
9,2019-05-09,00:14:00,中国证券报,上证180公司治理交易型开放式指数证券投资基金（更新）招募说明书摘要,上证180公司治理交易型开放式指数证券投资基金（更新）招募说明书摘要 中国证券报 ...,https://finance.sina.com.cn/stock/relnews/us/2...
